In [1]:
import pandas

train_ds = pandas.read_csv("youtube/PM_youtube_train.csv")
test_ds = pandas.read_csv("youtube/PM_youtube_test.csv")

In [2]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()
label_enc.fit(train_ds['subgroup'])
y_train = label_enc.transform(train_ds['subgroup']) 
y_test = label_enc.transform(test_ds['subgroup'])

y_train += 1
y_test += 1

In [3]:
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.pipeline import Pipeline, make_pipeline

X_train = train_ds.as_matrix()[:,5:].astype(float)
X_test = test_ds.as_matrix()[:,5:].astype(float)

mm = make_pipeline(MinMaxScaler(), Normalizer())
mm.fit(X_train)

X_train = mm.transform(X_train)
X_test = mm.transform(X_test)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

for d in range(3,15,1):
    for f in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]:
        model = RandomForestClassifier(n_estimators=100, max_features=f,max_depth=d,criterion='entropy',class_weight='balanced')
        scores = cross_val_score(model, X_train, y_train, scoring='f1_macro', cv=5, n_jobs=-1)
        print (d,f,np.mean(scores), np.std(scores))

In [ ]:
from sklearn.metrics import f1_score
model = RandomForestClassifier(n_estimators=100, max_features=0.5,max_depth=14,criterion='entropy',class_weight='balanced')
model.fit(X_train,y_train)

pred_y = model.predict(X_test)
scores = f1_score(pred_y,y_test,average=None)
print (np.mean(scores), np.std(scores))

In [ ]:
scores

In [ ]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

for d in range(2,5,1):
    model = XGBClassifier(max_depth=d)
    scores = cross_val_score(model, X_train, y_train, scoring='f1_macro', cv=5, n_jobs=-1)
    print (d,scores,np.mean(scores), np.std(scores))

In [ ]:
from sklearn.metrics import f1_score
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=5,use_label_encoder=False)
model.fit(X_train,y_train)

pred_y = model.predict(X_test)
scores = f1_score(pred_y,y_test,average=None)
print (np.mean(scores), np.std(scores))

In [ ]:
scores

# RBF SVM

In [ ]:
import numpy as np
#from thundersvm import *
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC

#for gamma in [100.]:
gamma = 0
for c in [5000]:
    kf = KFold(n_splits=5, shuffle=True)
    itr = 0
    res = []
    res2 = []
    for train, test in kf.split(X_train):       
        X_train_ = X_train[train]
        X_test_ = X_train[test]
        y_train_ = y_train[train]
        y_test_ = y_train[test]        
        cl = SVC(C=c,kernel='poly',tol=0.0001,max_iter=1000000,degree=4,verbose=False,class_weight = 'balanced')
        #cl = LinearSVC(C=c,tol=0.0001,max_iter=1000000, class_weight = 'balanced')

        cl.fit(X_train_,y_train_)#,sample_weights=deltas)
        y_pred = cl.predict(X_test_)
        y_pred2 = cl.predict(X_train_)
        res.append(f1_score(y_test_,y_pred,average=None)) 
        res2.append(f1_score(y_train_,y_pred2,average=None)) 
        print (res,res2)
        

    res = np.asarray(res)   
    res2 = np.asarray(res2) 
    print(gamma,c,res.mean(),res.std())
    print(gamma,c,res2.mean(),res2.std())


In [ ]:
0 3000 0.8355805345573482 0.10631811197292429
0 3000 0.9076560062560901 0.022493988147552536

In [ ]:
res

# Train my own KF

In [ ]:
import numpy as np
import numpy
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import f1_score
import CO2_tree as co2t
import CO2_forest as co2f
from scipy.sparse import csr_matrix
from sklearn.model_selection import KFold

tree_deth = [2,3,4]
sratios = [0.]
sratios2 = [1.0]
fratios = [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

C = [5000]



for d in tree_deth:
    for sratio in sratios:
        for sratio2 in sratios2:
            for fratio in fratios:
                for gm in [5]:
                    for ns in [0.]:                
                        kf = KFold(n_splits=5, shuffle=True)
                        itr = 0
                        res = []
                        res2 = []
                        for train, test in kf.split(X_train):       
                            X_train_ = csr_matrix(X_train[train])
                            X_test_ = csr_matrix(X_train[test])
                            y_train_ = y_train[train]
                            y_test_ = y_train[test]   
                            
                            
                            #cl = SVC(C=c,kernel='rbf',tol=0.0001,max_iter=1000,gamma=gamma,verbose=False)
                            cl = co2f.CO2_forest(C=5000, dual=False,tol = 0.0001,max_iter=100000000,kernel='polynomial',max_deth=d,n_jobs=5,sample_ratio=1.0, feature_ratio = fratio,n_estimators=100,gamma=gm,dropout_low=sratio,dropout_high=sratio2,noise=ns,cov_dr=0.,criteria='gain')
                            #cl = LinearSVC(C=c,tol=0.0001,max_iter=1000)

                            cl.fit(X_train_,y_train_)
                            y_pred = cl.predict(X_test_)
                            y_pred2 = cl.predict(X_train_)
                            res.append(f1_score(y_test_,y_pred,average=None)) 
                            res2.append(f1_score(y_train_,y_pred2,average=None)) 
                            #print (res,res2)

                        res = numpy.asarray(res)   
                        res2 = numpy.asarray(res2) 
                        print("Test:",d,gm,fratio,res.mean(),res.std())
                        print("Train:", d,gm,fratio,res2.mean(),res2.std())
                        #print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import nlopt
import numpy
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import CO2_tree as co2t
import CO2_forest as co2f
from scipy.sparse import csr_matrix
from joblib import Parallel, delayed

results = []

for i in range(20):
    print (i)
    cl = co2f.CO2_forest(C=5000, dual=False,tol = 0.0001,max_iter=2000000000,kernel='polynomial',max_deth=3,n_jobs=10,sample_ratio=1.0, feature_ratio = 0.3,n_estimators=10,gamma=5,dropout_low=0.0,dropout_high=1.0,noise=0.,cov_dr=100.,criteria='gain')
    cl.fit(csr_matrix(X_train), y_train, csr_matrix(X_test),y_test)
    y_pred_train = cl.predict(csr_matrix(X_train))
    y_pred = cl.predict(csr_matrix(X_test))
    rs_local_test = accuracy_score(y_test,y_pred)
    rs_local_train = accuracy_score(y_train,y_pred_train)
    #results.append([probs,r2,r3,rs_local_train - rs_local_test])

0


/workspace/notebooks/thunder/thundersvm/CO2_forest.py:217: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  r2 = cov(numpy.hstack(cprobs[:,:,layer] for layer in range(1,cprobs.shape[2])))


LR orig test result: 0.057558945908460446 C: 0.1 r2 before: 4.250286347637379 r2 after: 3.416959254332953
LR noised test result: 0.05963938973647709 C: 0.1 noise: 0.2 r2 before: 4.250286347637379 r2 after: 2.9604719741914853
LR balanced noise test result: 0.05894590846047154 C: 1.0 noise: 0.3 r2 before: 4.250286347637379 r2 after: 2.5449787395965333
LR naive pruning test result: 0.061026352288488184 C: 1.0 pruning: 0.1 r2 before: 4.250286347637379 r2 after: 2.9735177613459434
LR l2-norm pruning test result: 0.05963938973647709 C: 0.1 pruning: 0.4 r2 before: 4.250286347637379 r2 after: 3.318998572795651 instability before: 94.22940617247062 instability after: 9.21100065827613


In [ ]:
import nlopt
import numpy
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import CO2_tree as co2t
import CO2_forest as co2f
from scipy.sparse import csr_matrix
from joblib import Parallel, delayed

for _ in range(30):
    cl = co2f.CO2_forest(C=5000, dual=False,tol = 0.0001,max_iter=2000000000,kernel='polynomial',max_deth=3,n_jobs=10,sample_ratio=1.0, feature_ratio = 0.3,n_estimators=10,gamma=5,dropout_low=0.0,dropout_high=1.0,noise=0.,cov_dr=100.,criteria='gain')
    probs, r2,r3 = cl.fit(csr_matrix(X_train), y_train, csr_matrix(X_test),y_test)
    ratios = (1. / probs).sum(axis=1)
    idx = numpy.argmax(ratios)
    y_pred_train = cl.predict(csr_matrix(X_train))
    y_pred = cl.predict(csr_matrix(X_test))
    rs_local_test = accuracy_score(y_test,y_pred)
    rs_local_train = accuracy_score(y_train,y_pred_train) 
    print ("Drop:", idx)
    print (rs_local_train - rs_local_test)
    cl.trees.pop(idx)
    y_pred_train = cl.predict(csr_matrix(X_train))
    y_pred = cl.predict(csr_matrix(X_test))
    rs_local_test = accuracy_score(y_test,y_pred)
    rs_local_train = accuracy_score(y_train,y_pred_train)    
    print (rs_local_train - rs_local_test)
    

In [ ]:
(1. / probs).sum(axis=1)

In [ ]:
probs

In [ ]:
y_pred_train = cl.predict(csr_matrix(X_train))
y_pred = cl.predict(csr_matrix(X_test))
rs_local_test = accuracy_score(y_test,y_pred)
rs_local_train = accuracy_score(y_train,y_pred_train)

In [ ]:
rs_local_train - rs_local_test

In [ ]:
cl.trees.pop(0)

In [ ]:
y_pred_train = cl.predict(csr_matrix(X_train))
y_pred = cl.predict(csr_matrix(X_test))
rs_local_test = accuracy_score(y_test,y_pred)
rs_local_train = accuracy_score(y_train,y_pred_train)

In [ ]:
rs_local_train - rs_local_test

In [ ]:
rs_local_test

In [ ]:
import pickle

with open('results.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
import matplotlib.pyplot as plt

x = []
y = []

for r in results:
    x.append((r[0].sum(axis=0)*r[0].sum(axis=0)).sum())
    y.append(r[3])

plt.scatter(x, y)

plt.show()

In [ ]:
import matplotlib.pyplot as plt

x = []
y = []

for r in results:
    x.append((r[0]*r[0]).sum())
    y.append(r[3])

plt.scatter(x, y)

plt.show()

In [ ]:
import matplotlib.pyplot as plt

x = []
y = []

for r in results:
    x.append(r[1])
    y.append(r[3])

plt.scatter(x, y)

In [ ]:
import matplotlib.pyplot as plt

x = []
y = []

for r in results:
    x.append(r[2])
    y.append(r[3])

plt.scatter(x, y)

In [ ]:
import matplotlib.pyplot as plt

x = []
y = []

for r in results:
    x.append((1. / r[0]).sum())
    y.append(r[3])

plt.scatter(x, y)

In [ ]:
import matplotlib.pyplot as plt

x = []
y = []

for r in results:
    x.append((1. / r[0].sum(axis=0)).sum())
    y.append(r[3])

plt.scatter(x, y)

In [ ]:
import matplotlib.pyplot as plt

x = []
y = []

for r in results:
    x.append((1. / r[0]).sum() * r[1])
    y.append(r[3])

plt.scatter(x, y)

In [ ]:
import nlopt
import numpy
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import CO2_tree as co2t
import CO2_forest as co2f
from scipy.sparse import csr_matrix
from joblib import Parallel, delayed

cl = co2f.CO2_forest(C=5000, dual=False,tol = 0.0001,max_iter=2000000000,kernel='polynomial',max_deth=3,n_jobs=2,sample_ratio=1.0, feature_ratio = 0.3,n_estimators=2,gamma=5,dropout_low=0.0,dropout_high=1.0,noise=0.,cov_dr=100.,criteria='gain')

def genRatios(rest,step, depth):
    if depth > 1:
        if rest > 0:
            for val in numpy.arange(0.,rest+ 0.000001,step):
                #print("(",rest,val,depth,")")
                for rest_ in genRatios(rest - val,step,depth - 1):
                    yield [val] + rest_ 
        else:
            yield []
    else:
        yield [rest]

def weighter(tree,forest,w):
    t = forest.trees[tree]
    t.estimateChunkWeights(w[tree])
    return t

cl.fit(csr_matrix(X_train), y_train)
ratios_ = numpy.zeros((2,))
for i in numpy.arange(0,1.05,0.05):
    ratios_[0] = i
    ratios_[1] = 1. - i
    cl.trees = Parallel(n_jobs=cl.n_jobs,backend="multiprocessing")(delayed(weighter)(i,cl,ratios_) for i in range(cl.n_estimators))
    y_pred = cl.predict(csr_matrix(X_test))
    rs_local = accuracy_score(y_test,y_pred)
    print (i,1. - i, 1. - rs_local)

In [ ]:
global_max_f

In [ ]:
global_max_w

In [ ]:
#cov matrix and biases
from numpy import corrcoef
from sklearn.preprocessing import normalize

r_ = numpy.zeros((len(cl.trees), len(cl.trees))) 
for i in range(len(cl.trees)):
    for j in range(len(cl.trees)): 
        t1 = cl.trees[i].getCounts()
        t2 = cl.trees[j].getCounts()
        if len(t1.shape) != 2: 
            for k in range(t1.shape[0]): 
                if t1[k].shape[0] < 3*X_train.shape[0]:
                    tmp = numpy.zeros((3*X_train.shape[0],))
                    tmp[:t1[k].shape[0]] = t1[k]
                    t1[k] = tmp    
            t1 = numpy.vstack(t1)        
        if len(t2.shape) != 2:
            for k in range(t2.shape[0]): 
                if t2[k].shape[0] < 3*X_train.shape[0]:
                    tmp = numpy.zeros((3*X_train.shape[0],))
                    tmp[:t2[k].shape[0]] = t2[k]
                    t2[k] = tmp              
            t2 = numpy.vstack(t2)
        
        counts = numpy.vstack([t1, t2])
        corr = corrcoef(counts)

        tmp = []
        for k in range(corr.shape[0]):
            for l in range(corr.shape[1]):
                if (k > len(t1) and l < len(t1)) or (k < len(t1) and l > len(t1)):
                    tmp.append(corr[k,l])
        if i != j:            
            r_[i,j] = numpy.asarray(tmp).mean()            

probs = []
for t in cl.trees:
    probs.append(numpy.abs(numpy.asarray(t.getProbs()) - 1. / pow(2, cl.max_deth)).mean())
biases = numpy.asarray(probs)    
#all_counts = numpy.vstack(counts)
#r_ = corrcoef(all_counts)
#r_ = []
#counter_h = 0

#for i in range(len(counts)):
#    rl = {}
#    for j in range(len(counts[i])): 
#        counter_w = 0
#        for k in range(len(counts)):
#            rline = []
#            for l in range(len(counts[k])):
#                rline.append(r[counter_h, counter_w])
#                counter_w += 1
#            if k not in rl:
#                rl[k] = []
#            rl[k].append(rline)
#        counter_h += 1
#    tmp = []    
#    for k in range(len(counts)):
#        tmp.append(numpy.concatenate(rl[k]).mean())
#    r_.append(tmp)    
#r_ = numpy.asarray(r_)                
print ('biases')
print (biases)

r_[r_==1.0] = 0.
print ('r')
print(r_)

P = cl.get_err_matrix(csr_matrix(X_train), y_train,use_weight=False)            
G = numpy.dot(P,numpy.transpose(P))
G = G / G.max()

print ("G:",G)


In [ ]:
r2 = numpy.zeros((len(cl.trees), len(cl.trees))) 
for i in range(len(cl.trees)):
    for j in range(len(cl.trees)): 
        t1 = cl.trees[i].getCounts()
        t2 = cl.trees[j].getCounts()

        if len(t1.shape) != 2: 
            for k in range(t1.shape[0]): 
                if t1[k].shape[0] < 3*X_train.shape[0]:
                    tmp = numpy.zeros((3*X_train.shape[0],))
                    tmp[:t1[k].shape[0]] = t1[k]
                    t1[k] = tmp    
            t1 = numpy.vstack(t1)        
        if len(t2.shape) != 2:
            for k in range(t2.shape[0]): 
                if t2[k].shape[0] < 3*X_train.shape[0]:
                    tmp = numpy.zeros((3*X_train.shape[0],))
                    tmp[:t2[k].shape[0]] = t2[k]
                    t2[k] = tmp              
            t2 = numpy.vstack(t2)

        t1 = t1[:,:X_train.shape[0]] + t1[:,X_train.shape[0]:2*X_train.shape[0]] + t1[:,2*X_train.shape[0]:]
        t2 = t2[:,:X_train.shape[0]] + t2[:,X_train.shape[0]:2*X_train.shape[0]] + t2[:,2*X_train.shape[0]:] 
        
        #t1[t1 > 1] = 1.
        #t2[t2 > 1] = 1.
        
        counts = numpy.vstack([t1, t2])
        corr = corrcoef(counts)

        tmp = []
        for k in range(corr.shape[0]):
            for l in range(corr.shape[1]):
                if (k > len(t1) and l < len(t1)) or (k < len(t1) and l > len(t1)):
                    tmp.append(corr[k,l])
        if i != j:            
            r2[i,j] = numpy.asarray(tmp).mean()  

In [ ]:
r2[r2 < 0]

In [ ]:
from scipy.stats import pearsonr

pearsonr(global_max_w,biases)

In [ ]:
pearsonr(global_max_w,r_.mean(axis=1))

In [ ]:
pearsonr(global_max_w,r2.sum(axis=1))

In [ ]:
pearsonr(global_max_w,P.sum(axis=1))

In [ ]:
pearsonr(global_max_w,G.sum(axis=1))

In [ ]:
for t in cl.trees:
    print(numpy.asarray(t.getProbs()))

In [ ]:
(cl.trees[5].getCounts())[0].shape

In [ ]:
(cl.trees[5].getCounts())[3].shape

In [ ]:
f1_score(y_test,y_pred,average=None)
#4/poly/0.5 array([0.95306859, 0.88      , 0.95329874])

In [ ]:
print(f1_score(y_train,y_predt,average=None))

In [ ]:
array([0.95286439, 0.88590604, 0.95427729]) (3, poly, deg 4)

In [ ]:
np.max(X_train,axis=1)

In [ ]:
y_train_.shape